In [101]:
# import modules
import cv2
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from math import sqrt

In [102]:
class Calibrator:
    def __init__(self, images_path, image_size, pattern_size, square_size, flags_calib):
        # Calib config
        self.pattern_size = pattern_size
        self.square_size = square_size
        self.images_path = images_path
        self.image_size = image_size
        self.flags_calib = flags_calib

        # Data points
        self.world_points = []
        self.image_points = []

        # Calib results
        self.results = {}
    
    def calibrate(self):
        
        # Generate board points
        board_points = np.zeros((self.pattern_size[0] * self.pattern_size[1], 3), np.float32)
        board_points[:,:2] = np.mgrid[ 0:self.pattern_size[0], 0:self.pattern_size[1] ].T.reshape(-1,2)
        board_points = board_points * self.square_size
        
        # Detect corners in chessboard
        counter = 0
        for fname in self.images_path:
            img = cv2.imread(fname)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            self.img_size = gray.shape[::-1]
            detected, corners = cv2.findChessboardCornersSB(gray, self.pattern_size, None)
            if detected:
                self.world_points.append(board_points)
                self.image_points.append(corners)
                counter+=1

        print("Corners found in " + str(counter) + " images")

        # Calibrate the camera
        rms, camera_matrix, distortion_coeffs, rvecs, tvecs, std_intrinsic, std_extrinsic, per_view_error = cv2.calibrateCameraExtended(self.world_points, self.image_points, self.image_size, None, None,
                                                                flags=self.flags_calib)

        print("RMS re-projection error:", rms)
        print("The median re-projection error", np.median(per_view_error))
        print("Camera Matrix:\n", camera_matrix)
        print("Distortion Parameters:\n", distortion_coeffs)
        
        # Save the calibration results
        self.results = {
        "error_rms": rms,
        "camera_matrix": camera_matrix,
        "distortion_coeffs": distortion_coeffs,
        "rvecs": rvecs,
        "tvecs": tvecs,
        "std_intrinsic": std_intrinsic,
        "std_extrinsic": std_extrinsic,
        "per_view_error": per_view_error
        }

        return 

In [109]:
# Choose the image path
path = '../../../images/virtual/'
images = glob(path + '7x7/opengl3/*.jpg')

counter = 0
image_set = []

i = np.random.choice(np.arange(0,49),10, replace=False)

for idx in i:

    image_set.append(images[idx])

# Choose the calibration flags
flagsCalib = cv2.CALIB_RATIONAL_MODEL

In [110]:
myCameraCalbrator = Calibrator(images_path = image_set, image_size = (720,720), pattern_size = (7,7), square_size = 30, flags_calib = flagsCalib)

myCameraCalbrator.calibrate()

Corners found in 10 images
RMS re-projection error: 0.21789591699177224
The median re-projection error 0.22209601480882984
Camera Matrix:
 [[623.39518643   0.         360.05035132]
 [  0.         623.40300492 360.14082681]
 [  0.           0.           1.        ]]
Distortion Parameters:
 [[ 1.18321049e+00  2.03836462e+01 -9.00301121e-05  6.63275030e-04
  -1.08706670e+02  1.18235808e+00  2.03345460e+01 -1.08544316e+02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
